# 04_test_retrieval
Phase 1 testing for chunking, embedding, retrieval, and generation

In [ ]:
import json
from src.core.chunking import MarkdownChunker

chunker = MarkdownChunker()
chunks = []
with open("data/processed/youmed_articles_test.jsonl", "r", encoding="utf-8") as f:
    for line in f:
        doc = json.loads(line)
        chunks.extend(chunker.chunk_document(doc["content"], doc["metadata"]))

print(f"Chunks: {len(chunks)}")

In [ ]:
from src.core.embedding import ONNXEmbedding

embedding_model = ONNXEmbedding("models/bge-m3-onnx")
embeddings = embedding_model.embed_batch([c.enriched_content for c in chunks])
print(embeddings.shape)

In [ ]:
from src.db.vector_store import QdrantStore

store = QdrantStore(url="http://localhost:6333")
store.create_collection(name="youmed_articles", dimension=embeddings.shape[1])
store.upsert(chunks, embeddings)

In [ ]:
from src.core.retriever import BM25Retriever, HybridRetriever
from src.core.reranker import ONNXReranker

bm25 = BM25Retriever(chunks)
reranker = ONNXReranker("models/bge-reranker-v2-m3-onnx")
retriever = HybridRetriever(store, bm25, embedding_model, reranker)

query = "Benh Addison co trieu chung gi?"
results = retriever.retrieve(query, top_k=5)
for i, chunk in enumerate(results, 1):
    print(i, chunk.metadata.get("title"), chunk.original_content[:120])

In [ ]:
from src.core.generator import MedicalRAGGenerator

generator = MedicalRAGGenerator(openai_api_key="YOUR_API_KEY")
answer = generator.generate(query, results)
print(answer["answer"])